In [1]:
!gdown https://drive.google.com/uc?id=1Fxv32_ConX1dw3RldPEBrvPSbibvafeh

Downloading...
From: https://drive.google.com/uc?id=1Fxv32_ConX1dw3RldPEBrvPSbibvafeh
To: /content/Activity_Measurements_reduced.zip
100% 353M/353M [00:03<00:00, 96.2MB/s]


In [2]:
!unzip Activity_Measurements_reduced.zip

Archive:  Activity_Measurements_reduced.zip
replace Activity_Measurements/GOTOV05-GENEActiv-ankle.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Activity_Measurements/GOTOV05-GENEActiv-ankle.csv  
  inflating: Activity_Measurements/GOTOV05-GENEActiv-chest.csv  
  inflating: Activity_Measurements/GOTOV05-GENEActiv-wrist.csv  
  inflating: Activity_Measurements/GOTOV06-GENEActiv-ankle.csv  
  inflating: Activity_Measurements/GOTOV06-GENEActiv-chest.csv  
  inflating: Activity_Measurements/GOTOV06-GENEActiv-wrist.csv  
  inflating: Activity_Measurements/GOTOV07-GENEActiv-ankle.csv  
  inflating: Activity_Measurements/GOTOV07-GENEActiv-chest.csv  
  inflating: Activity_Measurements/GOTOV07-GENEActiv-wrist.csv  
  inflating: Activity_Measurements/GOTOV08-GENEActiv-ankle.csv  
  inflating: Activity_Measurements/GOTOV08-GENEActiv-chest.csv  
  inflating: Activity_Measurements/GOTOV08-GENEActiv-wrist.csv  
  inflating: Activity_Measurements/GOTOV09-GENEActiv-ankle.csv  
  inflating:

In [3]:
!pip install pyts

In [27]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from pyts.image import GramianAngularField
import numpy as np
import os

def read_and_normalize_data(sensor_type, folder_path):
    sensor_data = pd.DataFrame()
    num = 1
    for filename in os.listdir(folder_path):
        if filename.endswith(f"-GENEActiv-{sensor_type}.csv"):
            file_path = os.path.join(folder_path, filename)
            user_name = filename.split('-')[0]

            # Read CSV file
            df = pd.read_csv(file_path)

            # Add user name as a column and concatenate to the sensor_data DataFrame
            df['user'] = user_name
            new_data = pd.concat([pd.DataFrame(df['x']),pd.DataFrame(df['y']),pd.DataFrame(df['z']),pd.DataFrame(df['labels']),pd.DataFrame(df['user'])],axis=1)
            sensor_data = pd.concat([sensor_data,new_data],axis=0)
            num+=1
            if num>9:
              break
    return sensor_data

# Function to create GAF images for activity recognition
def create_gaf_images(sensor_data, window_size):
    images = []
    labels = []  # Assuming labels are present in the original dataset

    unique_users = sensor_data['user'].unique()
    print(unique_users)
    train_users = unique_users[0:-2]
    val_users = unique_users[-2]
    test_users = unique_users[-1]
    # Combine all data for normalization
    combined_data = pd.DataFrame()

    # training data
    for user in train_users:
        user_data = sensor_data[sensor_data['user'] == user]
        combined_data = pd.concat([combined_data, user_data])

    # Extract features and labels
    features = combined_data.iloc[:, 0:-2].values  # Exclude 'user' and 'labels' columns

    # Normalize the entire dataset
    scaler = StandardScaler()
    normalized_features = scaler.fit_transform(features)

    start_index = 0
    for user in unique_users:
        user_data = sensor_data[sensor_data['user'] == user]


            # Check if all labels in the window are the same

        while start_index + window_size <= len(user_data):
            label = user_data.iloc[start_index]['labels']  # Use the label of the first entry in the window
            if user_data.iloc[start_index:start_index + window_size]['labels'].nunique() == 1:
             # Extract and reshape window data for each column (x, y, z)
              curr_img = np.zeros((window_size,window_size,3))
              for col_index in range(normalized_features.shape[1]):
                  col_window = normalized_features[start_index:start_index + window_size, col_index]
                  reshaped_col_window = col_window.reshape(1, window_size)

                  # Create GAF images using GASF for each column
                  gasf = GramianAngularField()
                  image = gasf.fit_transform(reshaped_col_window)
                  curr_img[:,:,col_index] = image
                  # Add each channel image to the list
              images.append(curr_img)
              labels.append(user_data.iloc[start_index]['labels'])

            start_index += window_size

    images = np.array(images)
    labels = np.array(labels)



    return images, labels





# Main script
folder_path = "/content/Activity_Measurements"
window_size = 100  # Adjust window size based on your requirements

# Read and normalize data for each sensor type
ankle_data = read_and_normalize_data("ankle", folder_path)
# chest_data = read_and_normalize_data("chest", folder_path)
# wrist_data = read_and_normalize_data("wrist", folder_path)

# Create windows for each sensor type
ankle_windows = create_gaf_images(ankle_data, window_size)
# chest_windows = create_gaf_images(chest_data, window_size)
# wrist_windows = create_gaf_images(wrist_data, window_size)



<ipython-input-27-2f558618b5a8>:16: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


['GOTOV31' 'GOTOV25' 'GOTOV07' 'GOTOV05' 'GOTOV34' 'GOTOV10' 'GOTOV18'
 'GOTOV14' 'GOTOV16']


In [33]:
len(np.unique(ankle_windows[1]))

17

In [29]:
ankle_windows[0].shape

(3389, 100, 100, 3)

In [20]:
ankle_windows[2].shape

(3389, 100, 100, 3)

In [35]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_array = label_encoder.fit_transform(ankle_windows[1])


In [36]:
encoded_array

array([ 4, 11, 11, ...,  0,  0,  0])

In [37]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import numpy as np
image_size = 100
num_classes = 17
# Assuming you have a dataset with images and corresponding labels
# X_train: array of images, y_train: array of labels
# Ensure that your images and labels are properly loaded

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(ankle_windows[0], encoded_array, test_size=0.2, random_state=42)

# Define the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size, image_size, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))  # Adjust num_classes based on your dataset

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')


Epoch 1/10
85/85 [==============================] - 55s 635ms/step - loss: 2.0730 - accuracy: 0.3914 - val_loss: 1.7087 - val_accuracy: 0.4440
Epoch 2/10
85/85 [==============================] - 51s 593ms/step - loss: 1.5548 - accuracy: 0.4994 - val_loss: 1.6785 - val_accuracy: 0.4499
Epoch 3/10
85/85 [==============================] - 50s 585ms/step - loss: 1.4379 - accuracy: 0.5253 - val_loss: 1.6032 - val_accuracy: 0.4735
Epoch 4/10
85/85 [==============================] - 52s 614ms/step - loss: 1.3337 - accuracy: 0.5485 - val_loss: 1.5906 - val_accuracy: 0.4749
Epoch 5/10
85/85 [==============================] - 50s 586ms/step - loss: 1.2306 - accuracy: 0.5843 - val_loss: 1.5820 - val_accuracy: 0.5074
Epoch 6/10
85/85 [==============================] - 52s 616ms/step - loss: 1.1316 - accuracy: 0.6145 - val_loss: 1.6711 - val_accuracy: 0.4912
Epoch 7/10
85/85 [==============================] - 52s 612ms/step - loss: 1.0181 - accuracy: 0.6477 - val_loss: 1.6213 - val_accuracy: 0.5000